Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [8]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [9]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [10]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [11]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [13]:
batch_size = 128
hiden_layer_num=1024
wd = 0.001

tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(None, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  ## add relu layer >>
  relu_weights= tf.Variable(tf.truncated_normal([image_size*image_size,hiden_layer_num]))
  relu_biases = tf.Variable(tf.zeros([hiden_layer_num]))
  relu_layer = tf.nn.relu(tf.matmul(tf_train_dataset,relu_weights,name="mat_layer1")+relu_biases)
  ## add relu layer <<
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([hiden_layer_num, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(relu_layer, weights,name="mat_logits") + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)+
    tf.nn.l2_loss(weights)*wd + tf.nn.l2_loss(relu_weights)*wd
  )
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
#   valid_prediction = tf.nn.softmax(
#     tf.matmul(tf_valid_dataset, weights) + biases)
#   test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [14]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        train_prediction.eval(feed_dict={tf_train_dataset:valid_dataset}),valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(train_prediction.eval(feed_dict={tf_train_dataset:test_dataset}), test_labels))

Initialized
Minibatch loss at step 0: 675.353760
Minibatch accuracy: 9.4%
Validation accuracy: 36.7%
Minibatch loss at step 500: 205.872665
Minibatch accuracy: 77.3%
Validation accuracy: 77.0%
Minibatch loss at step 1000: 114.809273
Minibatch accuracy: 82.0%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 68.884613
Minibatch accuracy: 78.1%
Validation accuracy: 81.8%
Minibatch loss at step 2000: 41.312408
Minibatch accuracy: 86.7%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 25.389105
Minibatch accuracy: 81.2%
Validation accuracy: 84.4%
Minibatch loss at step 3000: 15.528667
Minibatch accuracy: 85.2%
Validation accuracy: 86.6%
Minibatch loss at step 3500: 9.553951
Minibatch accuracy: 89.8%
Validation accuracy: 87.0%
Minibatch loss at step 4000: 6.038664
Minibatch accuracy: 89.1%
Validation accuracy: 87.4%
Minibatch loss at step 4500: 3.773213
Minibatch accuracy: 90.6%
Validation accuracy: 87.3%
Minibatch loss at step 5000: 2.594439
Minibatch accuracy: 89.1%
Validatio

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [27]:
num_steps = 6001
batch_size = 128

train_dataset_10_batch = train_dataset[:1280,:]
train_labels_10_batch = train_labels[:1280,:]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_10_batch.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_10_batch[offset:(offset + batch_size), :]
    batch_labels = train_labels_10_batch[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        train_prediction.eval(feed_dict={tf_train_dataset:valid_dataset}),valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(train_prediction.eval(feed_dict={tf_train_dataset:test_dataset}), test_labels))

Initialized
Minibatch loss at step 0: 646.173645
Minibatch accuracy: 10.9%
Validation accuracy: 26.9%
Minibatch loss at step 500: 189.904907
Minibatch accuracy: 100.0%
Validation accuracy: 77.9%
Minibatch loss at step 1000: 115.126495
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 1500: 69.805870
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Minibatch loss at step 2000: 42.330292
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Minibatch loss at step 2500: 25.671038
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 3000: 15.570321
Minibatch accuracy: 100.0%
Validation accuracy: 78.4%
Minibatch loss at step 3500: 9.474189
Minibatch accuracy: 99.2%
Validation accuracy: 78.7%
Minibatch loss at step 4000: 5.742640
Minibatch accuracy: 100.0%
Validation accuracy: 79.1%
Minibatch loss at step 4500: 3.504114
Minibatch accuracy: 100.0%
Validation accuracy: 79.9%
Minibatch loss at step 5000: 2.151742
Minibatch accuracy: 100.0%

The above graph have L2 regulirization. let's try to remvoe L2. see that happens

In [28]:
batch_size = 128
hiden_layer_num=1024
wd = 0.0

tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(None, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  ## add relu layer >>
  relu_weights= tf.Variable(tf.truncated_normal([image_size*image_size,hiden_layer_num]))
  relu_biases = tf.Variable(tf.zeros([hiden_layer_num]))
  relu_layer = tf.nn.relu(tf.matmul(tf_train_dataset,relu_weights,name="mat_layer1")+relu_biases)
  ## add relu layer <<
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([hiden_layer_num, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(relu_layer, weights,name="mat_logits") + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)+
    tf.nn.l2_loss(weights)*wd + tf.nn.l2_loss(relu_weights)*wd
  )
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
#   valid_prediction = tf.nn.softmax(
#     tf.matmul(tf_valid_dataset, weights) + biases)
#   test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [29]:
num_steps = 6001
batch_size = 128

train_dataset_10_batch = train_dataset[:1280,:]
train_labels_10_batch = train_labels[:1280,:]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_10_batch.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_10_batch[offset:(offset + batch_size), :]
    batch_labels = train_labels_10_batch[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        train_prediction.eval(feed_dict={tf_train_dataset:valid_dataset}),valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(train_prediction.eval(feed_dict={tf_train_dataset:test_dataset}), test_labels))

Initialized
Minibatch loss at step 0: 316.709290
Minibatch accuracy: 6.2%
Validation accuracy: 42.2%
Minibatch loss at step 500: 0.000017
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
Minibatch loss at step 1000: 0.000085
Minibatch accuracy: 100.0%
Validation accuracy: 77.2%
Minibatch loss at step 1500: 0.000102
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 2000: 0.000143
Minibatch accuracy: 100.0%
Validation accuracy: 76.4%
Minibatch loss at step 2500: 0.000088
Minibatch accuracy: 100.0%
Validation accuracy: 76.2%
Minibatch loss at step 3000: 0.000058
Minibatch accuracy: 100.0%
Validation accuracy: 76.0%
Minibatch loss at step 3500: 0.106437
Minibatch accuracy: 99.2%
Validation accuracy: 74.4%
Minibatch loss at step 4000: 0.000026
Minibatch accuracy: 100.0%
Validation accuracy: 75.5%
Minibatch loss at step 4500: 0.019983
Minibatch accuracy: 99.2%
Validation accuracy: 75.4%
Minibatch loss at step 5000: 0.000129
Minibatch accuracy: 100.0%
Validatio

ok. validation and test accuracy is lower compared to with L2. but Minibath accuracy is still 100%. so, looks like L2 actually works!

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [36]:
batch_size = 128
hiden_layer_num=1024
wd = 0.001

tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(None, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  ## add relu layer >>
  relu_weights= tf.Variable(tf.truncated_normal([image_size*image_size,hiden_layer_num]))
  relu_biases = tf.Variable(tf.zeros([hiden_layer_num]))
  ## add relu layer <<
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([hiden_layer_num, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  def model(training=False):
    relu_layer = tf.nn.relu(tf.matmul(tf_train_dataset,relu_weights,name="mat_layer1")+relu_biases)
    if training:
      relu_layer = tf.nn.dropout(relu_layer,0.5)
    logits = tf.matmul(relu_layer, weights,name="mat_logits") + biases
    return logits
  training_logits = model(True)
  # Training computation.
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=training_logits)+
    tf.nn.l2_loss(weights)*wd + tf.nn.l2_loss(relu_weights)*wd
  )
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(training_logits)
  valid_prediction = tf.nn.softmax(model(False))
#   valid_prediction = tf.nn.softmax(
#     tf.matmul(tf_valid_dataset, weights) + biases)
#   test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [37]:
num_steps = 6001
batch_size = 128

train_dataset_10_batch = train_dataset[:1280,:]
train_labels_10_batch = train_labels[:1280,:]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_10_batch.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_10_batch[offset:(offset + batch_size), :]
    batch_labels = train_labels_10_batch[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict={tf_train_dataset:valid_dataset}),valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(valid_prediction.eval(feed_dict={tf_train_dataset:test_dataset}), test_labels))

Initialized
Minibatch loss at step 0: 893.633301
Minibatch accuracy: 7.8%
Validation accuracy: 31.4%
Minibatch loss at step 500: 196.927048
Minibatch accuracy: 96.1%
Validation accuracy: 79.3%
Minibatch loss at step 1000: 116.836014
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 1500: 70.860214
Minibatch accuracy: 100.0%
Validation accuracy: 80.3%
Minibatch loss at step 2000: 43.007915
Minibatch accuracy: 99.2%
Validation accuracy: 80.3%
Minibatch loss at step 2500: 26.070549
Minibatch accuracy: 100.0%
Validation accuracy: 80.5%
Minibatch loss at step 3000: 15.823730
Minibatch accuracy: 99.2%
Validation accuracy: 80.5%
Minibatch loss at step 3500: 9.630049
Minibatch accuracy: 98.4%
Validation accuracy: 80.3%
Minibatch loss at step 4000: 5.830911
Minibatch accuracy: 100.0%
Validation accuracy: 80.7%
Minibatch loss at step 4500: 3.558235
Minibatch accuracy: 100.0%
Validation accuracy: 80.9%
Minibatch loss at step 5000: 2.189966
Minibatch accuracy: 100.0%
Val

a bitter higher than without droput

try to training model on whole training dataset instead of 10 batches

In [38]:
num_steps = 6001
batch_size = 128

# train_dataset_10_batch = train_dataset[:1280,:]
# train_labels_10_batch = train_labels[:1280,:]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict={tf_train_dataset:valid_dataset}),valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(valid_prediction.eval(feed_dict={tf_train_dataset:test_dataset}), test_labels))

Initialized
Minibatch loss at step 0: 803.168823
Minibatch accuracy: 6.2%
Validation accuracy: 28.8%
Minibatch loss at step 500: 224.235382
Minibatch accuracy: 73.4%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 116.247223
Minibatch accuracy: 76.6%
Validation accuracy: 80.2%
Minibatch loss at step 1500: 69.337418
Minibatch accuracy: 71.9%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 41.823990
Minibatch accuracy: 75.0%
Validation accuracy: 82.4%
Minibatch loss at step 2500: 25.526062
Minibatch accuracy: 81.2%
Validation accuracy: 83.8%
Minibatch loss at step 3000: 15.632753
Minibatch accuracy: 78.9%
Validation accuracy: 85.7%
Minibatch loss at step 3500: 9.629525
Minibatch accuracy: 81.2%
Validation accuracy: 86.1%
Minibatch loss at step 4000: 6.179338
Minibatch accuracy: 82.0%
Validation accuracy: 86.2%
Minibatch loss at step 4500: 3.874179
Minibatch accuracy: 87.5%
Validation accuracy: 86.2%
Minibatch loss at step 5000: 2.641924
Minibatch accuracy: 84.4%
Validatio

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
